### The purpose of this notebook is to answer question one of my analysis questions:

#### How many marijuana-involved incidents did the SFPD report each year over time?

In [1]:
#import modules
import pandas as pd
import altair as alt

/Users/nbaustin/.pyenv/versions/3.8.5/envs/sf_crime_data_analysis-3.8.5/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Import our cleaned dataset that contains all of our marijuana incidents. We made this .csv file in the 01_data_cleaning notebook.

In [2]:
mari_incidents = pd.read_csv('all_data_marijuana.csv', dtype=str)

Convert our incident dates to a datetime data format.

In [3]:
mari_incidents['incident_date'] = pd.to_datetime(mari_incidents['incident_date'])

Check our date ranges

In [4]:
mari_incidents['incident_date'].min()

Timestamp('2003-01-01 00:00:00')

In [5]:
mari_incidents['incident_date'].max()

Timestamp('2021-10-09 00:00:00')

Looks like we've got a full year of data for 2003, our earliest year. But since 2021 ends in October, we can't do full annual analysis on that year. So let's make a dataframe with our full years of data.

In [6]:
full_years = mari_incidents[
    (mari_incidents['incident_date'] >= '2003-01-01') &
    (mari_incidents['incident_date'] < '2021-01-01')
].reset_index(drop=True)

We know from our data dictionary that there are multiple row entries for some individual incidents. But we also know that the incident_number will remain the same across all entries related to the same incident. So since we're just looking at how many incidents there were in each year, we can go ahead and drop all the duplicates in the incident_number column. (Data dictionary is here: https://datasf.gitbook.io/datasf-dataset-explainers/sfpd-incident-report-2018-to-present)

In [7]:
full_years_incidents = full_years.drop_duplicates(subset=['incident_number'])

Now we want to get annual counts of incidents using groupby

In [8]:
annual_mari_incidents = full_years_incidents.groupby([pd.Grouper(key='incident_date', axis=0, freq='A')]).count()

In [9]:
annual_mari_incidents

,row_id,incident_number,incident_code,incident_category,incident_description,day_of_week,incident_time,police_district,resolution,longitude,latitude,the_geom
incident_date,,,,,,,,,,,,
2003-12-31,1701,1701,1701,1701,1701,1701,1701,1701,1701,1701,1701,1701
2004-12-31,1579,1579,1579,1579,1579,1579,1579,1579,1579,1579,1579,1579
2005-12-31,1125,1125,1125,1125,1125,1125,1125,1125,1125,1125,1125,1125
2006-12-31,1216,1216,1216,1216,1216,1216,1216,1216,1216,1216,1216,1216
2007-12-31,1636,1636,1636,1636,1636,1636,1636,1636,1636,1636,1636,1636
2008-12-31,1880,1880,1880,1880,1880,1880,1880,1880,1880,1880,1880,1880
2009-12-31,1973,1973,1973,1973,1973,1973,1973,1973,1973,1973,1973,1973
2010-12-31,1654,1654,1654,1654,1654,1654,1654,1654,1654,1654,1654,1654
2011-12-31,1086,1086,1086,1086,1086,1086,1086,1086,1086,1086,1086,1086


Now let's subset for just one column, incident_number, and reset the index so that incident data becomes a new column. 

In [10]:
annual_mari_incidents = annual_mari_incidents[['incident_number']].reset_index()

In [11]:
annual_mari_incidents

,incident_date,incident_number
0,2003-12-31,1701
1,2004-12-31,1579
2,2005-12-31,1125
3,2006-12-31,1216
4,2007-12-31,1636
5,2008-12-31,1880
6,2009-12-31,1973
7,2010-12-31,1654
8,2011-12-31,1086
9,2012-12-31,1073


Now let's create a year column

In [12]:
annual_mari_incidents['year'] = annual_mari_incidents['incident_date'].dt.year

In [13]:
annual_mari_incidents

,incident_date,incident_number,year
0,2003-12-31,1701,2003
1,2004-12-31,1579,2004
2,2005-12-31,1125,2005
3,2006-12-31,1216,2006
4,2007-12-31,1636,2007
5,2008-12-31,1880,2008
6,2009-12-31,1973,2009
7,2010-12-31,1654,2010
8,2011-12-31,1086,2011
9,2012-12-31,1073,2012


Now let's trim down our columns, rename them, and reorder them.

In [14]:
annual_mari_incidents.rename(columns={'incident_number': 'marijuana_incidents'}, inplace=True)
annual_mari_incidents

,incident_date,marijuana_incidents,year
0,2003-12-31,1701,2003
1,2004-12-31,1579,2004
2,2005-12-31,1125,2005
3,2006-12-31,1216,2006
4,2007-12-31,1636,2007
5,2008-12-31,1880,2008
6,2009-12-31,1973,2009
7,2010-12-31,1654,2010
8,2011-12-31,1086,2011
9,2012-12-31,1073,2012


In [15]:
annual_mari_incidents_final = annual_mari_incidents[['year', 'marijuana_incidents']].copy()

In [16]:
annual_mari_incidents_final

,year,marijuana_incidents
0,2003,1701
1,2004,1579
2,2005,1125
3,2006,1216
4,2007,1636
5,2008,1880
6,2009,1973
7,2010,1654
8,2011,1086
9,2012,1073


### Time to visualize!

In [17]:
alt.Chart(annual_mari_incidents_final).mark_bar().encode(
    x='year:O',
    y='marijuana_incidents'
).properties(
    title='San Francisco Police: Annual Number of Marijuana Incidents'
)

alt.Chart(...)